In [2]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [3]:
# 1. Obtener un set de datos.
df = pd.read_csv("data.csv")

# 2. Buscar valores vacios
print("Missing values:")
print(df.isna().sum())

# 3. Eliminar columna que no agrega valor al entrenamiento
df = df.drop('Employee_ID', axis=1)

Missing values:
Employee_ID                      0
Age                              0
Gender                           0
Marital_Status                   0
Department                       0
Job_Role                         0
Job_Level                        0
Monthly_Income                   0
Hourly_Rate                      0
Years_at_Company                 0
Years_in_Current_Role            0
Years_Since_Last_Promotion       0
Work_Life_Balance                0
Job_Satisfaction                 0
Performance_Rating               0
Training_Hours_Last_Year         0
Overtime                         0
Project_Count                    0
Average_Hours_Worked_Per_Week    0
Absenteeism                      0
Work_Environment_Satisfaction    0
Relationship_with_Manager        0
Job_Involvement                  0
Distance_From_Home               0
Number_of_Companies_Worked       0
Attrition                        0
dtype: int64


In [4]:
# Guardar las columnas categóricas
categorical_cols = ['Gender', 'Marital_Status', 'Department', 'Job_Role', 'Overtime']

# Guardar las columnas numéricas
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['Attrition']]

In [5]:
# 4. Separar features y target
X = df.drop('Attrition', axis=1)
y = df['Attrition']

In [6]:
# 5. Dividir en train y test con un split 80 - 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 6. Aplicar las técnicas de escalamiento y transformación de datos
# Transformar datos categóricos mediante One Hot Encoder
encoder = OneHotEncoder(sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])

# Escalamiento de datos numéricos mediante StandarScaler
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])

In [8]:
# 7. Obtener nombres de features
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Crear DataFrames para datos categóricos transformados
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=encoded_feature_names)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, columns=encoded_feature_names)

# Crear DataFrames para datos numéricos transformados
X_train_num_df = pd.DataFrame(X_train_num_scaled, columns=numerical_cols)
X_test_num_df = pd.DataFrame(X_test_num_scaled, columns=numerical_cols)

In [9]:
# 8. Combinar datos categóricos y numéricos
X_train_processed = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_processed = pd.concat([X_test_num_df, X_test_cat_df], axis=1)

In [10]:
# 9. Transformar target a valor numerico mediante LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [11]:
# 8. Implementación de Modelo 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

input_shape = X_train_processed.shape[1:]

model = Sequential([
    Flatten(input_shape=input_shape),  
    Dense(256, activation="relu"),  
    Dense(1, activation="sigmoid")   
])

/Users/sebastianflores/myenv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# 9. Compilamos modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
# 10. Entrenamiento de modelo
history = model.fit(
    X_train_processed, y_train_encoded,
    epochs=50,
    validation_data=(X_test_processed, y_test_encoded)
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 914us/step - accuracy: 0.7672 - loss: 0.5391 - val_accuracy: 0.7960 - val_loss: 0.5123
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.8054 - loss: 0.4903 - val_accuracy: 0.7960 - val_loss: 0.5202
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.8025 - loss: 0.4918 - val_accuracy: 0.7960 - val_loss: 0.5171
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.8009 - loss: 0.4817 - val_accuracy: 0.7960 - val_loss: 0.5165
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.7982 - loss: 0.4801 - val_accuracy: 0.7960 - val_loss: 0.5180
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.8061 - loss: 0.4712 - val_accuracy: 0.7955 - val_loss: 0.5241
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - accuracy: 0.7981 - loss: 0.4720 - val_accuracy: 0.7950 - val_loss: 0.5309
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8054 - loss: 0.4607 - 

In [36]:
# 11. Evaluación inicial del modelo  
test_loss, test_acc = model.evaluate(X_test_processed, y_test_encoded)
print(f'Accuracy: {test_acc:.4f}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - accuracy: 0.7386 - loss: 0.6948
Accuracy: 0.7415


In [39]:
predictions = model.predict(X_test_processed)
classes_x = np.argmax(predictions,axis=1)

classes_x

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step


array([0, 0, 0, ..., 0, 0, 0])

In [40]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(classes_x)):
  if classes_x[i] == 1:
    if y_test_encoded[i] == 1:
      TP = TP + 1
    else :
      FP = FP + 1
  else:
    if y_test_encoded[i] == 0:
      TN = TN + 1
    else :
      FN = FN + 1


print('         ', 'label neg ', ' label pos')
print('pred neg    ', TN, "        ", FN)
print('pred pos    ', FP, "        ", TP)

          label neg   label pos
pred neg     1592          408
pred pos     0          0
